In [11]:
import sys
sys.path.append("/Users/yuzhezhang/Desktop/FinancialEngineering/C++")
import math
from scipy.stats import norm
import bs  
import QuantLib as ql

In [ ]:


class EuropeanOption:
    def __init__(self, S, K, r, T, sigma):
        if S <= 0 or K <= 0 or r < 0 or T <= 0 or sigma <= 0:
            raise ValueError("Invalid parameters for European Option")
        self.S = S
        self.K = K
        self.r = r
        self.T = T
        self.sigma = sigma
        self.d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
        self.d2 = self.d1 - sigma * math.sqrt(T)

    def price_call(self):
        return self.S * norm.cdf(self.d1) - self.K * math.exp(-self.r * self.T) * norm.cdf(self.d2)

    def price_put(self):
        return self.K * math.exp(-self.r * self.T) * norm.cdf(-self.d2) - self.S * norm.cdf(-self.d1)

    def delta_call(self):
        return norm.cdf(self.d1)

    def delta_put(self):
        return norm.cdf(self.d1) - 1

    def gamma(self):
        return norm.pdf(self.d1) / (self.S * self.sigma * math.sqrt(self.T))

    def vega(self):
        return self.S * norm.pdf(self.d1) * math.sqrt(self.T)

    def theta_call(self):
        part1 = -(self.S * norm.pdf(self.d1) * self.sigma) / (2 * math.sqrt(self.T))
        part2 = self.r * self.K * math.exp(-self.r * self.T) * norm.cdf(self.d2)
        return part1 - part2

    def theta_put(self):
        part1 = -(self.S * norm.pdf(self.d1) * self.sigma) / (2 * math.sqrt(self.T))
        part2 = self.r * self.K * math.exp(-self.r * self.T) * norm.cdf(-self.d2)
        return part1 + part2

    def rho_call(self):
        return self.K * self.T * math.exp(-self.r * self.T) * norm.cdf(self.d2)

    def rho_put(self):
        return -self.K * self.T * math.exp(-self.r * self.T) * norm.cdf(-self.d2)


In [10]:
%%time
py_test = EuropeanOption(100, 100, 0.05, 1.0, 0.2)
py_test.price_call()

CPU times: user 352 μs, sys: 66 μs, total: 418 μs
Wall time: 394 μs


np.float64(10.450583572185565)

In [7]:
%%time
cpp_test = bs.EuropeanOption(100, 100, 0.05, 1.0, 0.2)
cpp_test.priceCall()

CPU times: user 44 μs, sys: 0 ns, total: 44 μs
Wall time: 47.2 μs


10.450583572185565

In [12]:
today = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = today
spot_price = 100     
strike_price = 100   
interest_rate = 0.05 
volatility = 0.2     
maturity = 1.0   
expiry_date = today + int(365 * maturity)    
calendar = ql.TARGET()
day_count = ql.Actual365Fixed()
spot = ql.QuoteHandle(ql.SimpleQuote(spot_price))
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, interest_rate, day_count))
vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, calendar, volatility, day_count))
bsm_process = ql.BlackScholesProcess(spot, flat_ts, vol_ts)

In [19]:
%%time
european_exercise = ql.EuropeanExercise(expiry_date)
payoff_call = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)
option_call = ql.VanillaOption(payoff_call, european_exercise)
engine = ql.AnalyticEuropeanEngine(bsm_process)
option_call.setPricingEngine(engine)
option_call.NPV()

CPU times: user 144 μs, sys: 0 ns, total: 144 μs
Wall time: 147 μs


10.450583572185575